# Интеллектуальные методы обработки видео

## Модуль 3


In [3]:
# Константы
dataFolder = "./data"

### Чтение данных


In [17]:
# Загрузка видео в память и вырезание фрагмента
from moviepy.editor import VideoFileClip

src_file_name = f"{dataFolder}/src/vid1.avi"
result_file_name = f"{dataFolder}/result/temp_cut_video.avi"

# Загрузить видеофайл
video = VideoFileClip(src_file_name)
# Вырезать часть видео с 10 по 20 секунду
cut_video = video.subclip(10, 25)
# Сохранить вырезанный фрагмент в новый файл
cut_video.write_videofile(result_file_name, codec="libx264", preset="ultrafast", bitrate="5000k")

Moviepy - Building video ./data/optical_flow_stabilized_compared_min.mp4.
MoviePy - Writing audio in optical_flow_stabilized_compared_minTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./data/optical_flow_stabilized_compared_min.mp4



Moviepy - Done !
Moviepy - video ready ./data/optical_flow_stabilized_compared_min.mp4


In [1]:
# Загрузка видео из Youtube
from pytube import YouTube

# URL видео, которое вы хотите скачать
video_url = "https://www.youtube.com/watch?v=MjViy6kyiqs"

try:
    # Создание объекта YouTube
    yt = YouTube(video_url)

    # Получение первого потока (обычно это наивысшее качество видео)
    video_stream = yt.streams.get_highest_resolution()

    # Скачивание видео
    video_stream.download(output_path=f"{dataFolder}/result/")

    print("Видео успешно скачано и сохранено в", f"{dataFolder}/result/")
except Exception as e:
    print("Ошибка:", e)

Видео успешно скачано и сохранено в ./data


In [22]:
# Используем opencv-python для чтения временного файла и сохранения его в формате FFV1:
import cv2
import os
from moviepy.editor import VideoFileClip

src_file_name = result_file_name
result_file_name = f"{dataFolder}/result/final_cut_video.avi"

# Инициализировать объект VideoCapture для чтения временного файла
cap = cv2.VideoCapture(src_file_name)

# Получить параметры видео для создания VideoWriter
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Инициализировать VideoWriter с кодеком FFV1
fourcc = cv2.VideoWriter_fourcc(*"FFV1")
out = cv2.VideoWriter(result_file_name, fourcc, fps, (frame_width, frame_height), True)

# Читать каждый кадр из временного файла и записывать его в новый файл с кодеком FFV1
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)

# Освободить ресурсы
cap.release()
out.release()

# Удаление временного файла, если необходимо
os.remove(src_file_name)

Moviepy - Building video ./data/temp_cut_video.avi.
MoviePy - Writing audio in temp_cut_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./data/temp_cut_video.avi



Moviepy - Done !
Moviepy - video ready ./data/temp_cut_video.avi


In [5]:
# Чтение свойств файлов
import cv2

result_file_name = f"{dataFolder}/result/final_cut_video.avi"


def print_video_info(file_path):
    # Открыть видеофайл
    cap = cv2.VideoCapture(file_path)

    if not cap.isOpened():
        print("Ошибка при открытии файла")
        return

    # Чтение параметров видео
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    codec = int(cap.get(cv2.CAP_PROP_FOURCC))
    codec_str = "".join([chr((codec >> 8 * i) & 0xFF) for i in range(4)])
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps if fps > 0 else 0

    # Вывод информации
    print("\n")
    print(f"Файл: {file_path}")
    print(f"Разрешение: {int(width)}x{int(height)}")
    print(f"Кодек: {codec_str}")
    print(f"Количество кадров: {frame_count}")
    print(f"FPS (Кадров в секунду): {fps}")
    print(f"Продолжительность (секунды): {duration:.2f}")

    cap.release()


# Пример использования
print_video_info(result_file_name)



Файл: ./data/result/final_cut_video.avi
Разрешение: 608x448
Кодек: ffv1
Количество кадров: 250
FPS (Кадров в секунду): 25.0
Продолжительность (секунды): 10.00


### Используем метод оптического потока для стабилизации видео


In [6]:
import cv2
import numpy as np

src_file_name = f"{dataFolder}/result/final_cut_video.avi"
result_file_name = f"{dataFolder}/result/optical_flow.avi"


def is_black_frame(frame, threshold=10):
    """Проверяет, является ли кадр черным."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return np.mean(gray_frame) < threshold


def stabilize_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)

    # Чтение первого кадра
    ret, prev = cap.read()
    if not ret:
        print("Не удалось прочитать видео")
        cap.release()
        return

    # Преобразование в градации серого
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    # Параметры для детектора углов ShiTomasi
    feature_params = dict(maxCorners=100, qualityLevel=0.03, minDistance=7, blockSize=7)

    # Параметры для оптического потока Лукаса-Канаде
    lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Выбор точек для отслеживания
    p0 = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

    # Создание случайных цветов
    color = np.random.randint(0, 255, (100, 3))

    # Параметры для записи видео
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (prev.shape[1], prev.shape[0]))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if is_black_frame(frame):
            out.write(frame)  # Сохраняем оригинальный черный кадр
            # Переинициализируем точки после черного экрана
            p0 = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
            prev_gray = frame_gray
            continue

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Вычисление оптического потока
        p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, p0, None, **lk_params)

        if p1 is not None:
            # Выбор хороших точек
            good_new = p1[st == 1]
            good_old = p0[st == 1]
        else:
            # Использование последних известных хороших точек
            good_new = good_old

        # Рисование треков
        # Рисование треков
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            frame = cv2.line(frame, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

        img = cv2.add(frame, np.zeros(np.shape(frame), dtype=np.uint8))
        out.write(img)

        # Обновление предыдущего кадра и предыдущих точек
        prev_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    cap.release()
    out.release()
    cv2.destroyAllWindows()


# Используйте функцию как stabilize_video('путь_к_исходному_видео', 'путь_к_стабилизированному_видео')
stabilize_video(src_file_name, result_file_name)

In [7]:
# Итог
import cv2
import numpy as np

src_file_name = f"{dataFolder}/result/final_cut_video.avi"
result_file_name = f"{dataFolder}/result/optical_flow_stabilized.avi"


def is_black_frame(frame, threshold=10):
    """Проверяет, является ли кадр черным."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return np.mean(gray_frame) < threshold


def stabilize_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    ret, prev = cap.read()
    if not ret:
        print("Не удалось прочитать видео")
        cap.release()
        return
    fps = cap.get(cv2.CAP_PROP_FPS)
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    feature_params = dict(maxCorners=100, qualityLevel=0.03, minDistance=7, blockSize=7)
    lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    p0 = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_path, fourcc, fps, (prev.shape[1], prev.shape[0]))

    # Инициализация накопительной матрицы преобразований
    cumulative_trans = np.float32([[1, 0, 0], [0, 1, 0]])

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if is_black_frame(frame):
            out.write(frame)
            p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)
            prev_gray = frame_gray
            continue

        p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, p0, None, **lk_params)

        if p1 is not None and st.any():
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Вычисление среднего смещения
            displacements = good_new - good_old
            dx_avg = np.mean(displacements[:, 0])
            dy_avg = np.mean(displacements[:, 1])

            # Обновление накопительной матрицы преобразований
            cumulative_trans[0, 2] += -dx_avg
            cumulative_trans[1, 2] += -dy_avg

            # Применение скорректированного смещения
            frame_stabilized = cv2.warpAffine(frame, cumulative_trans, (frame.shape[1], frame.shape[0]))

            out.write(frame_stabilized)
        else:
            out.write(frame)

        prev_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2) if p1 is not None else p0

    cap.release()
    out.release()
    cv2.destroyAllWindows()


stabilize_video(src_file_name, result_file_name)

### Используем готовый алгоритм stabilize библиотеки vidstab для стабилизации видео


In [9]:
from vidstab import VidStab



src_file_name = f"{dataFolder}/result/final_cut_video.avi"
result_file_name = f"{dataFolder}/result/stabilized_file.avi"



# Инициализировать стабилизатор видео


stabilizer = VidStab()



# Стабилизировать исходное видео


stabilizer.stabilize(input_path=src_file_name, output_path=result_file_name, output_fourcc="FFV1")



# Вывести сообщение об успешной стабилизации


print(f"Видео стабилизировано и сохранено в файл {result_file_name}")

Видео стабилизировано и сохранено в файл 'stabilized_file.avi'


> Однако в результате мы получаем проблему с переходом через черный экран
> Чтобы решить эту проблему придется разбить видео на части, стабилизировать все фрагменты, кроме черного экрана.


In [12]:
import cv2
from vidstab import VidStab

src_file_name = f"{dataFolder}/result/final_cut_video.avi"
result_file_name = f"{dataFolder}/result/final_stabilized_video.avi"


def is_black_frame(frame, threshold=10):
    """Проверяет, является ли кадр черным."""
    return cv2.mean(frame)[0] < threshold


def find_video_segments(input_path):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    frames = []
    start_frame = None
    is_current_frame_black = False
    previous_frame_black = False

    current_frame = 0
    while True:
        ret, frame = cap.read()
        if not ret:  # Видео закончилось
            if start_frame is not None and current_frame - 1 != start_frame:
                # Добавляем последний сегмент, если он не заканчивается черным экраном
                frames.append((start_frame, current_frame - 1, previous_frame_black))
            break

        is_current_frame_black = is_black_frame(frame)

        if start_frame is None:
            # Начало нового сегмента
            start_frame = current_frame
            previous_frame_black = is_current_frame_black
        elif is_current_frame_black != previous_frame_black:
            # Конец текущего сегмента и начало нового
            frames.append((start_frame, current_frame - 1, previous_frame_black))
            start_frame = current_frame
            previous_frame_black = is_current_frame_black

        current_frame += 1

    cap.release()
    return frames, fps


def save_video_segments(input_path, segments):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    temp_files = []
    is_black_segments = []  # Список для отслеживания, является ли сегмент черным экраном

    for i, (start_frame, end_frame, is_black) in enumerate(segments):
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        segment_filename = f"{dataFolder}/temp/pre_segment_{i}.avi"
        temp_files.append(segment_filename)
        is_black_segments.append(is_black)  # Сохраняем информацию о типе сегмента

        if is_black:
            # Для черных экранов просто сохраняем один кадр, так как не требуется стабилизация
            ret, frame = cap.read()
            height, width, layers = frame.shape
            out = cv2.VideoWriter(segment_filename, fourcc, fps, (width, height))
            out.write(frame)  # Записываем один кадр для сохранения времени сегмента
        else:
            # Для обычных сегментов сохраняем весь диапазон кадров
            out = None
            for _ in range(end_frame - start_frame + 1):
                ret, frame = cap.read()
                if not ret:
                    break
                if out is None:
                    height, width, layers = frame.shape
                    out = cv2.VideoWriter(segment_filename, fourcc, fps, (width, height))
                out.write(frame)

        if out is not None:
            out.release()

    cap.release()
    return temp_files, is_black_segments  # Возвращаем информацию о файле и о том, является ли сегмент черным экраном


def stabilize_and_merge(segments, final_output, fps, input_path):
    stabilizer = VidStab()
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = None

    for i, (start_frame, end_frame, is_black) in enumerate(segments):
        if is_black:
            # Для черных сегментов просто копируем их в итоговое видео
            cap = cv2.VideoCapture(input_path)
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            for _ in range(end_frame - start_frame + 1):
                ret, frame = cap.read()
                if not ret:
                    break
                if out is None:
                    height, width, layers = frame.shape
                    out = cv2.VideoWriter(final_output, fourcc, fps, (width, height))
                out.write(frame)
            cap.release()
        else:
            # Стабилизируем сегмент
            segment_path = f"{dataFolder}/temp/pre_segment_{i}.avi"
            stabilized_path = segment_path.replace("pre", "stabilized")
            stabilizer.stabilize(input_path=segment_path, output_path=stabilized_path)

            # Добавляем стабилизированный сегмент в итоговое видео
            cap = cv2.VideoCapture(stabilized_path)
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                if out is None:
                    height, width, layers = frame.shape
                    out = cv2.VideoWriter(final_output, fourcc, fps, (width, height))
                out.write(frame)
            cap.release()

    if out is not None:
        out.release()


# Использование
video_segments, fps = find_video_segments(src_file_name)  # Пример сегментов, замените этими значениями реальные сегменты
temp_files = save_video_segments(src_file_name, video_segments)
stabilize_and_merge(video_segments, result_file_name, fps, src_file_name)

> Обратите внимание, final_cut_video - 204 мб, stabilized_file - 18 мб. Оба файла в форматах без потери качества: rawvideo и FFV1 соответственно.


### Полезные преобразования


In [17]:
# Сопоставить видео
from moviepy.editor import VideoFileClip, clips_array


def comparevideo(path1, path2, path3):
    # Загрузить оба видеофайла
    video1 = VideoFileClip(path1)
    video2 = VideoFileClip(path2)

    # Убедиться, что оба видео имеют одинаковую высоту
    video1_resized = video1.resize(height=video2.h)

    # Создать массив из видео для горизонтального объединения
    final_clip = clips_array([[video1_resized, video2]])

    # Экспортировать итоговое видео
    final_clip.write_videofile(path3, codec="libx264")


# comparevideo("final_cut_video.avi", "optical_flow_stabilized.avi", "optical_flow_stabilized_compared.mp4")
# comparevideo("final_cut_video.avi", "final_stabilized_video.avi", "final_stabilized_video_compared.mp4")

In [18]:
src_files_folder = f"{dataFolder}/src"
comparevideo(f"{src_files_folder}/01_fg_1.mp4", f"{src_files_folder}/01_bg_1.mp4", f"{dataFolder}/result/compare.mp4")

Moviepy - Building video ./data/result/src.mp4.
Moviepy - Writing video ./data/result/src.mp4



Moviepy - Done !
Moviepy - video ready ./data/result/src.mp4


In [19]:
# Факультативно. Стабилизируем видеопоток для публичного примера
from vidstab import VidStab, download_ostrich_video

path = f"{dataFolder}/src/ostrich.mp4"
download_ostrich_video(path)
stabilizer = VidStab()
stabilizer.stabilize(path, f"{dataFolder}/result/ostrich_stabilized.avi")

In [20]:
# Сохраним видео в animated gif
from moviepy.editor import VideoFileClip


def videoToGif(fname, fps=15):
    # Загрузить видеофайл
    video = VideoFileClip(f"{dataFolder}{fname}")
    # Сохранить вырезанный фрагмент в виде GIF с частотой кадров 15 fps
    video.write_gif(f"{dataFolder}{''.join(fname.split('.')[:-1])}.gif", fps=fps)

In [9]:
videoToGif("Mating3.mp4")

MoviePy - Building file ./data/Mating3.gif with imageio.


## Фильтрация шума


In [21]:
import cv2
import numpy as np

src_file_name = f"{dataFolder}/result/final_stabilized_video.avi"
result_file_name = f"{dataFolder}/result/filtered_video.avi"

# Открытие исходного видео
cap = cv2.VideoCapture(src_file_name)

# Получение параметров исходного видео
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

# Создание объекта записи видео
out = cv2.VideoWriter(result_file_name, cv2.VideoWriter_fourcc(*"DIVX"), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Применение фильтров к кадру
    blur = cv2.blur(frame, (5, 5))
    gaussian = cv2.GaussianBlur(frame, (5, 5), 0)
    median = cv2.medianBlur(frame, 5)

    # Объединение результатов фильтрации в одно изображение
    top_half = np.hstack((blur, gaussian))
    bottom_half = np.hstack((median, frame))
    combined_frame = np.vstack((top_half, bottom_half))

    # Уменьшаем размер комбинированного кадра до оригинального размера для сохранения
    combined_frame_resized = cv2.resize(combined_frame, (frame_width, frame_height))

    # Запись комбинированного кадра
    out.write(combined_frame_resized)

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()

### Методы заполнения на основе диффузии

Методы заполнения на основе диффузии используются для восстановления утраченных или поврежденных частей изображений путем постепенного распространения информации о текстуре и интенсивности из неповрежденных областей в области, подлежащие восстановлению. Один из наиболее известных методов диффузии - это алгоритм заполнения по Навье-Стоксу, основанный на концепции, используемой в компьютерной графике для синтеза текстур.

Прямая реализация таких алгоритмов может быть довольно сложной, особенно в контексте короткого примера кода, но я могу показать вам базовую иллюстрацию концепции диффузии на примере простой линейной диффузии для изображения. Это не будет полноценным методом заполнения на основе диффузии, но позволит вам понять основную идею.

Для более продвинутых методов, таких как заполнение по Навье-Стоксу, я бы рекомендовал обратиться к специализированным библиотекам и исследованиям в области обработки изображений.

В качестве примера рассмотрим простую диффузию, где каждый пиксель изображения обновляется на основе среднего значения его непосредственных соседей. Это может быть рассмотрено как базовая форма диффузии.


In [22]:
import cv2
import numpy as np


def diffuse_image(image, iterations=10):
    img = image.astype(float)
    for _ in range(iterations):
        # Вычисляем среднее значение соседних пикселей
        kernel = np.array([[0.05, 0.2, 0.05], [0.2, -1, 0.2], [0.05, 0.2, 0.05]])
        diffused_img = cv2.filter2D(img, -1, kernel) + img
        img = np.clip(diffused_img, 0, 255)
    return img.astype(np.uint8)


# Загрузка изображения
image_path = f"{dataFolder}/src/def1.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Применение простой диффузии
diffused_image = diffuse_image(image)

# Показать изображение
cv2.imwrite(f"{dataFolder}/result/def2.png", diffused_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
import cv2
import numpy as np

# Загрузка изображения
image = cv2.imread(f"{dataFolder}/src/def1.jpg")

# Создание маски для повреждённой области (предположим, что это белые пиксели)
mask = np.zeros(image.shape[:2], np.uint8)  # Создаём черную маску
# Допустим, у нас есть область с координатами [x, y, width, height], которую нужно восстановить
# Например:
x, y, width, height = 50, 50, 100, 100  # Координаты повреждённой области
mask[y : y + height, x : x + width] = 255  # Отмечаем повреждённую область на маске белым цветом

# Применение функции inpaint для заполнения повреждённых участков
inpaint_radius = 3  # Радиус каждого восстанавливаемого патча
inpaint_method = cv2.INPAINT_TELEA  # Можно использовать cv2.INPAINT_TELEA или cv2.INPAINT_NS
restored_image = cv2.inpaint(image, mask, inpaint_radius, inpaint_method)

# Показываем результаты
# cv2.imshow('Original Image', image)
# cv2.imshow('Mask', mask)
# cv2.imshow('Restored Image', restored_image)
cv2.imwrite(f"{dataFolder}/result/def3_mask.png", mask)
cv2.imwrite(f"{dataFolder}/result/def3_rest.png", restored_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Карты прозрачности


In [24]:
import cv2
import numpy as np

src_file_name1 = f"{dataFolder}/src/01_fg_1.mp4"
src_file_name2 = f"{dataFolder}/src/01_bg_1.mp4"
result_file_name = f"{dataFolder}/result/overlay_video.mp4"

# Параметры зеленого цвета для хромакея
green_lower = np.array([36, 25, 25])
green_upper = np.array([86, 255, 255])

# Открытие видеофайлов
cap_foreground = cv2.VideoCapture(src_file_name1)
cap_background = cv2.VideoCapture(src_file_name2)

# Получение параметров видео
frame_width = int(cap_background.get(3))
frame_height = int(cap_background.get(4))
fps = cap_background.get(cv2.CAP_PROP_FPS)

# Создание объекта для записи видео
out = cv2.VideoWriter(result_file_name, cv2.VideoWriter_fourcc(*"MP4V"), fps, (frame_width, frame_height))

while cap_foreground.isOpened() and cap_background.isOpened():
    ret_foreground, frame_foreground = cap_foreground.read()
    ret_background, frame_background = cap_background.read()

    if not ret_foreground or not ret_background:
        break

    # Изменение размера переднего плана, если необходимо, для соответствия размеру фона
    if frame_foreground.shape[:2] != frame_background.shape[:2]:
        frame_foreground = cv2.resize(frame_foreground, (frame_background.shape[1], frame_background.shape[0]))

    # Конвертация изображения в HSV и создание маски для зеленого фона
    hsv = cv2.cvtColor(frame_foreground, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, green_lower, green_upper)
    mask_inv = cv2.bitwise_not(mask)

    # Применение маски к переднему плану и фону
    foreground = cv2.bitwise_and(frame_foreground, frame_foreground, mask=mask_inv)
    background = cv2.bitwise_and(frame_background, frame_background, mask=mask)

    # Наложение изображений
    result = cv2.add(foreground, background)

    # Запись в файл
    out.write(result)

    cv2.imshow("Result", result)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap_foreground.release()
cap_background.release()
out.release()
cv2.destroyAllWindows()